# Develop Model Driver

In this notebook, we will develop the API that will call our model. This module initializes the model, transforms the input so that it is in the appropriate format and defines the scoring method that will produce the predictions. The API will expect the input to be in JSON format. Once a request is received, the API will use the request body to score the question text. There are two main functions in the API. The first function loads the model and returns a scoring function. The second function process the question text and uses the first function to score it.

In [1]:
import pandas as pd
import logging
import json

We use the writefile magic to write the contents of the below cell to driver.py which includes the driver methods.

In [2]:
%%writefile driver.py

import lightgbm as lgb
import timeit as t
import logging
from duplicate_model import DuplicateModel

model_path = 'model.pkl'
questions_path = 'questions.tsv'
logger = logging.getLogger("model_driver")

def create_scoring_func():
    """ Initialize Model Object 
    """   
    start = t.default_timer()
    DM = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    
    loadTimeMsg = "Model object loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)
    
    def call_model(text):
        preds = DM.score(text)  
        return preds
    
    return call_model

def get_model_api():
    logger = logging.getLogger("model_driver")
    scoring_func = create_scoring_func()
    
    def process_and_score(inputString):
        """ Classify the input using the loaded model
        """
        start = t.default_timer()

        responses = []
        preds = scoring_func(inputString)
        responses.append(preds)

        end = t.default_timer()
        
        logger.info("Predictions: {0}".format(responses))
        logger.info("Predictions took {0} ms".format(round((end-start)*1000, 2)))
        return (responses, "Computed in {0} ms".format(round((end-start)*1000, 2)))
    return process_and_score

def version():
    return lgb.__version__

Overwriting driver.py


Let's test the module.

In [3]:
logging.basicConfig(level=logging.DEBUG)

We run the driver.py which will bring the imports and functions into the context of the notebook.

In [4]:
%run driver.py

Now, let's use one of the duplicate questions to test our driver.

In [5]:
dupes_test_path = 'dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

'calulating length in javascript. below is my object in javascript, how can i calculate the length of it.   treeobj .length is not working. any help would be appreciated. thanks.'

Here, we define a helper function to convert our text for the format that will be required by the Flask application that will use the functions in the driver.

In [6]:
def text_to_json(text):
    return json.dumps({'input':'{0}'.format(text)})

In [7]:
jsontext = text_to_json(text_to_score)
json_load_text = json.loads(jsontext)
body = json_load_text['input']

In [8]:
predict_for = get_model_api()

INFO:model_driver:Model object loading time: 393.16 ms


In [9]:
resp = predict_for(body)

INFO:model_driver:Predictions: [[(5223, 6700, 0.9925665753453555), (126100, 4889658, 0.9912753937777351), (4616202, 4616273, 0.6155471701474984), (19590865, 19590901, 0.4620763251999151), (951021, 951057, 0.06970955925122703), (7364150, 7364307, 0.026884047907130657), (85992, 86014, 0.026080062267242667), (11922383, 11922384, 0.011795645104492705), (728360, 728694, 0.009326921967673533), (1470488, 1470494, 0.009134834316071474), (572897, 572996, 0.00883143996513768), (171251, 171256, 0.0076743796957673485), (149055, 149150, 0.00517983816744855), (1069666, 1069840, 0.004706627420234399), (695050, 695053, 0.004460205667197415), (1726630, 1726662, 0.003869085504916017), (1068834, 1144249, 0.003679768643276954), (364952, 364997, 0.0032878619315965516), (4057440, 4060176, 0.0022698985677602032), (4968406, 4968448, 0.0020199453774702746), (901115, 901144, 0.0014602612102322617), (610406, 610415, 0.0013977203916864665), (2100758, 2100767, 0.0013300039455512653), (1026069, 1026087, 0.001225461

Next, we move on to building our docker image.